In [1]:
import numpy as np
from src.utils import *
from tqdm import tqdm

unary_operators=[np.sin, np.cos, np.exp, np.abs, np.log, np.tan]
binary_operators=[np.add, np.subtract, np.multiply, np.divide]


operators = unary_operators + binary_operators

In [2]:
problem_0 = np.load("data/problem_4.npz")
x = problem_0["x"]
y = problem_0["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)

x.shape: (2, 5000)
y.shape: (5000,)


In [ ]:
NUM_POPULATION = 100
NUM_GENERATIONS = 1000
MAX_DEPTH_INITIAL = 5
DEDUPE_INTERVAL = 20
ELITISM = True
ELITE_COUNT = 3
TOURNAMENT_SUBSET_SIZE = 3
TOURNAMENT_WINNER_SIZE = 15
XOVER = 0.8
MUTATION = 0.8
BREED_NEW = 15
KILL_AGE = 15
CONSTANT_FIT_INTERVAL = 10
CONSTANT_FIT_ITERATION = 7

my_population = create_population(NUM_POPULATION,MAX_DEPTH_INITIAL,x.shape[0])
assign_population_fitness(my_population,x,y)
print(f" Population size: {len(my_population)}")
print(f"Population mean fitness: {calculate_mean_fitness(my_population)}")
print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")

# Sort the population by fitness (ascending), then take the top 5
top_5_individuals = top_n_individuals(my_population, 5)

# Print the top 5 individuals with the minimum fitness
for i, individual in enumerate(top_5_individuals, 1):
    print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")

In [ ]:
assign_population_fitness(my_population,x,y)
best_result = top_n_individuals(my_population, 1)[0]
for generation in tqdm(range(NUM_GENERATIONS)):
    num_xover=0
    num_mut=0
    num_succesfull_mut=0
    age_population(my_population)
    

    # Kill eldest individuals
    kill_eldest(my_population, KILL_AGE)
    # print("Population size before killing constants: ", len(my_population))
    
    # print("Population size after killing constants: ", len(my_population))


    best_ind = tournament_selection(my_population,TOURNAMENT_SUBSET_SIZE,TOURNAMENT_WINNER_SIZE,elitism=ELITISM, elite_count=ELITE_COUNT)

    
    for _ in range(BREED_NEW): # CHECK
        if random.random() < XOVER:
            # Xover the best individual and replace worst individuals
            parent1 = my_population[best_ind[np.random.choice(len(best_ind))]]
            parent2 = my_population[best_ind[np.random.choice(len(best_ind))]]
            child1, child2 = crossover(parent1,parent2)
            my_population.append(child1)
            my_population.append(child2)
            num_xover+=1

        if random.random() < MUTATION:
            # Mutate best individuals and replace worst individuals
            parent = my_population[best_ind[np.random.choice(len(best_ind))]]
            
            child, success = mutation_w_sa(parent, x.shape[0], x, y)
            num_mut+=1
            if success:                
                my_population.append(child)
                num_succesfull_mut+=1

    assign_population_fitness(my_population,x,y)
    # print(f"xover: {num_xover}, mut: {num_mut}, succesfull mut: {num_succesfull_mut}")
    simplify_constant_population(my_population)
    # Update elites age as 0
    elites = top_n_individuals(my_population, ELITE_COUNT)
    if best_result.fitness > elites[0].fitness:
        best_result = elites[0]
        print(f"New best result found: {best_result}")
    
    for elite in elites:
        my_population.remove(elite)
        elite.age = 0
        my_population.append(elite)

        
    
    # best_10 = top_n_individuals(my_population, 20)
    # for ind in best_10:
    #     print(f" Genome: {ind.genome}")

    # print("Population mean complexity: ", calculate_mean_complexity(my_population))
    
    # print("Population mean complezity after simplifying: ", calculate_mean_complexity(my_population))
    # best_10 = top_n_individuals(my_population, 20)
    # for ind in best_10:
    #     print(f" Genome: {ind.genome}")

    # Deduplicate every few iterations
    if generation % DEDUPE_INTERVAL == DEDUPE_INTERVAL-1:
        init_population = len(my_population)
        my_population = deduplicate_population(my_population)
        dedup_population = len(my_population)
        simplify_operation_population(my_population)
        simplified_operations = len(my_population)
        kill_constant(my_population)
        no_constant = len(my_population)

        print(f"Initial: {init_population}, Deduplicated: {init_population-dedup_population}, Operation simplified: {dedup_population-simplified_operations}, Constant: {simplified_operations-no_constant}, Final: {no_constant}")
        print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")
        print(f"Population mean fitness: {calculate_mean_fitness(my_population)}")
        print()
        best_5 = top_n_individuals(my_population, 5)
        for ind in range(len(best_5)):
            print(f"{ind}. Fitness: {best_5[ind].fitness:.3f}, Genome: {best_5[ind].genome}, Age: {best_5[ind].age}")
    #     # Fit constants of every individual
    # if generation % CONSTANT_FIT_INTERVAL == CONSTANT_FIT_INTERVAL-1:
    #     print(f"Generation {generation}: Mean fitness {calculate_mean_fitness(my_population)}")
    #     for individual in my_population:
    #         stronger_inv = fit_constants(individual,CONSTANT_FIT_ITERATION, x, y)
    #         my_population.remove(individual)
    #         my_population.append(stronger_inv)
    #     print(f"Population mean fitness after constant fit: {calculate_mean_fitness(my_population)}")
        
my_population = deduplicate_population(my_population)

In [ ]:
assign_population_fitness(my_population,x,y)
# Sort the population by fitness (ascending), then take the top 5
top_5_individuals = top_n_individuals(my_population, 15)

# Print the best individual
print(f"Best individual: Fitness = {best_result.fitness}: Genome = {best_result.genome}")
# Print the top 5 individuals with the minimum fitness in population
for i, individual in enumerate(top_5_individuals, 1):
    print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")

# Print the mean fitness of the population
print("Mean fitness of the population: ", calculate_mean_fitness(my_population))
print("Population size: ", len(my_population))
print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")

In [5]:
no_constant_optimization = my_population

In [3]:
NUM_POPULATION = 100
NUM_GENERATIONS = 500
MAX_DEPTH_INITIAL = 5
DEDUPE_INTERVAL = 15
ELITISM = True
ELITE_COUNT = 3
TOURNAMENT_SUBSET_SIZE = 3
TOURNAMENT_WINNER_SIZE = 15
XOVER = 0.8
MUTATION = 0.8
BREED_NEW = 15
KILL_AGE = 15
CONSTANT_FIT_INTERVAL = 30
CONSTANT_FIT_ITERATION = 5

my_population = create_population(NUM_POPULATION,MAX_DEPTH_INITIAL,x.shape[0])
assign_population_fitness(my_population,x,y)
print(f" Population size: {len(my_population)}")
print(f"Population mean fitness: {calculate_mean_fitness(my_population)}")
print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")

# Sort the population by fitness (ascending), then take the top 5
top_5_individuals = top_n_individuals(my_population, 5)

# Print the top 5 individuals with the minimum fitness
for i, individual in enumerate(top_5_individuals, 1):
    print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")

 Population size: 66
Population mean fitness: 33735564.981372274
Population mean complexity: 29.393939393939394
Top 1: Fitness = 20.664600300131436: Genome = cos(x[1])
Top 2: Fitness = 21.648812423604546: Genome = abs([1.83604604])
Top 3: Fitness = 21.796561472266546: Genome = abs([1.58198251])
Top 4: Fitness = 22.04313693206508: Genome = exp(abs(abs(exp(cos([1.59705876])))))
Top 5: Fitness = 22.46491656974902: Genome = ([-0.05771258] + [1.13939471])


In [4]:
assign_population_fitness(my_population,x,y)
best_result = top_n_individuals(my_population, 1)[0]
for generation in tqdm(range(NUM_GENERATIONS)):
    num_xover=0
    num_mut=0
    num_succesfull_mut=0
    age_population(my_population)
    

    # Kill eldest individuals
    kill_eldest(my_population, KILL_AGE)
    # print("Population size before killing constants: ", len(my_population))
    
    # print("Population size after killing constants: ", len(my_population))


    best_ind = tournament_selection(my_population,TOURNAMENT_SUBSET_SIZE,TOURNAMENT_WINNER_SIZE,elitism=ELITISM, elite_count=ELITE_COUNT)

    
    for _ in range(BREED_NEW): # CHECK
        if random.random() < XOVER:
            # Xover the best individual and replace worst individuals
            parent1 = my_population[best_ind[np.random.choice(len(best_ind))]]
            parent2 = my_population[best_ind[np.random.choice(len(best_ind))]]
            child1, child2 = crossover(parent1,parent2)
            my_population.append(child1)
            my_population.append(child2)
            num_xover+=1

        if random.random() < MUTATION:
            # Mutate best individuals and replace worst individuals
            parent = my_population[best_ind[np.random.choice(len(best_ind))]]
            
            child, success = mutation_w_sa(parent, x.shape[0], x, y)
            num_mut+=1
            if success:                
                my_population.append(child)
                num_succesfull_mut+=1

    assign_population_fitness(my_population,x,y)
    # print(f"xover: {num_xover}, mut: {num_mut}, succesfull mut: {num_succesfull_mut}")
    simplify_constant_population(my_population)
    # Update elites age as 0
    elites = top_n_individuals(my_population, ELITE_COUNT)
    if best_result.fitness > elites[0].fitness:
        best_result = elites[0]
        print(f"New best result found: {best_result}")
    
    for elite in elites:
        my_population.remove(elite)
        elite.age = 0
        my_population.append(elite)

        

    # Deduplicate every few iterations
    if generation % DEDUPE_INTERVAL == DEDUPE_INTERVAL-1:
        init_population = len(my_population)
        my_population = deduplicate_population(my_population)
        dedup_population = len(my_population)
        simplify_operation_population(my_population)
        simplified_operations = len(my_population)
        kill_constant(my_population)
        no_constant = len(my_population)

        print(f"Initial: {init_population}, Deduplicated: {init_population-dedup_population}, Operation simplified: {dedup_population-simplified_operations}, Constant: {simplified_operations-no_constant}, Final: {no_constant}")
        print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")
        print(f"Population mean fitness: {calculate_mean_fitness(my_population)}")
        print()
        best_5 = top_n_individuals(my_population, 5)
        for ind in range(len(best_5)):
            print(f"{ind}. Fitness: {best_5[ind].fitness:.3f}, Genome: {best_5[ind].genome}, Age: {best_5[ind].age}")
        # Fit constants of every individual
    # if generation % CONSTANT_FIT_INTERVAL == CONSTANT_FIT_INTERVAL-1:
    #     print(f"Generation {generation}: Mean fitness {calculate_mean_fitness(my_population)}")
    #     for individual in my_population:
    #         stronger_inv = fit_constants(individual,CONSTANT_FIT_ITERATION, x, y)
    #         my_population.remove(individual)
    #         my_population.append(stronger_inv)
    #     print(f"Population mean fitness after constant fit: {calculate_mean_fitness(my_population)}")
    #     best_5 = top_n_individuals(my_population, 5)
    #     for ind in range(len(best_5)):
    #         print(f"{ind}. Fitness: {best_5[ind].fitness:.3f}, Genome: {best_5[ind].genome}, Age: {best_5[ind].age}")
        
        
my_population = deduplicate_population(my_population)

  3%|▎         | 15/500 [04:12<2:09:42, 16.05s/it]

Initial: 581, Deduplicated: 372, Operation simplified: 0, Constant: 128, Final: 81
Population mean complexity: 49.79012345679013
Population mean fitness: 49621319.74481875

0. Fitness: 20.665, Genome: cos(x[1]), Age: 0
1. Fitness: 22.995, Genome: (log(abs(abs(([0.18298715] * x[0])))) + [3.70780283]), Age: 12
2. Fitness: 23.028, Genome: abs(sin(x[1])), Age: 11
3. Fitness: 23.132, Genome: cos(cos(([-1.26917945] - ([-0.98615547] + (x[1] * [-1.06500517]))))), Age: 15
4. Fitness: 23.155, Genome: cos(cos(x[0])), Age: 13


  6%|▌         | 30/500 [09:23<2:26:59, 18.76s/it]

Initial: 499, Deduplicated: 323, Operation simplified: 0, Constant: 70, Final: 106
Population mean complexity: 57.490566037735846
Population mean fitness: 7.897644106297515e+122

0. Fitness: 20.665, Genome: cos(x[1]), Age: 0
1. Fitness: 21.694, Genome: (cos(cos(x[0])) - [-1.43403746]), Age: 0
2. Fitness: 22.286, Genome: ([1.446616] + (cos(x[0]) + [1.2288712])), Age: 5
3. Fitness: 22.417, Genome: ([1.446616] + (cos(x[0]) + [0.15192654])), Age: 10
4. Fitness: 22.995, Genome: (log(abs(abs(([0.18298715] * x[0])))) + [3.70780283]), Age: 12


  9%|▉         | 45/500 [13:20<1:57:08, 15.45s/it]

...

Initial: 487, Deduplicated: 187, Operation simplified: 0, Constant: 62, Final: 238
Population mean complexity: 374.02100840336135
Population mean fitness: 3262205.9256538735

0. Fitness: 20.665, Genome: cos(x[1]), Age: 0
1. Fitness: 21.694, Genome: (cos(cos(x[0])) - [-1.43403746]), Age: 0
2. Fitness: 22.190, Genome: abs((cos(x[0]) + [1.8341821])), Age: 10
3. Fitness: 22.191, Genome: (sin(sin((x[0] - [0.65306623]))) + [1.446616]), Age: 2
4. Fitness: 22.673, Genome: abs(tan(cos((sin(abs(([-0.70418462] - x[0]))) * [0.97084835])))), Age: 3


 21%|██        | 105/500 [30:43<1:51:49, 16.99s/it]

Initial: 498, Deduplicated: 159, Operation simplified: 0, Constant: 62, Final: 277
Population mean complexity: 97.31768953068593
Population mean fitness: 2869.5312061689733

0. Fitness: 20.665, Genome: cos(x[1]), Age: 0
1. Fitness: 21.694, Genome: (cos(cos(x[0])) - [-1.43403746]), Age: 0
2. Fitness: 21.989, Genome: abs(tan(cos(([0.73040984] * (sin((sin(abs(((sin((x[0] - x[1])) - [1.446616]) - [0.15192654]))) - [0.15192654])) - [0.15192654]))))), Age: 7
3. Fitness: 22.176, Genome: abs(tan(cos(([0.73040984] * (sin(abs(((sin((x[0] - abs(sin(abs(((x[0] - tan(exp(sin(sin(((x[0] - (x[1] + [0.15192654])) - x[0])))))) - [0.82435966])))))) - [1.446616]) - [0.15192654]))) - [0.15192654]))))), Age: 5
4. Fitness: 22.190, Genome: abs((cos(x[0]) + [1.8341821])), Age: 14


 24%|██▍       | 119/500 [39:37<2:06:52, 19.98s/it]


TypeError: 'NoneType' object is not subscriptable

In [ ]:
assign_population_fitness(my_population,x,y)
# Sort the population by fitness (ascending), then take the top 5
top_5_individuals = top_n_individuals(my_population, 15)

# Print the best individual
print(f"Best individual: Fitness = {best_result.fitness}: Genome = {best_result.genome}")
# Print the top 5 individuals with the minimum fitness in population
for i, individual in enumerate(top_5_individuals, 1):
    print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")

# Print the mean fitness of the population
print("Mean fitness of the population: ", calculate_mean_fitness(my_population))
print("Population size: ", len(my_population))
print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")